In [1]:
import pandas as pd
import numpy as np
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.datasets import make_multilabel_classification
from skmultilearn.model_selection import IterativeStratification
# from sklearn.model_selection import StratifiedGroupKFold
import random
import csv
import glob
import sklearn
import os

/tmp/ipykernel_2256916/3352108578.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from collections import defaultdict

from sklearn.utils import (
    _approximate_mode,
    _safe_indexing,
    check_random_state,
    indexable,
    metadata_routing,
)

from sklearn.utils.validation import _num_samples, check_array, column_or_1d
from sklearn.utils.multiclass import type_of_target

In [3]:
class StratifiedGroupKFold_WithoutCheck(sklearn.model_selection._split.GroupsConsumerMixin, sklearn.model_selection._split._BaseKFold):
    """Stratified K-Fold iterator variant with non-overlapping groups.

    This cross-validation object is a variation of StratifiedKFold attempts to
    return stratified folds with non-overlapping groups. The folds are made by
    preserving the percentage of samples for each class.

    Each group will appear exactly once in the test set across all folds (the
    number of distinct groups has to be at least equal to the number of folds).

    The difference between :class:`~sklearn.model_selection.GroupKFold`
    and :class:`~sklearn.model_selection.StratifiedGroupKFold` is that
    the former attempts to create balanced folds such that the number of
    distinct groups is approximately the same in each fold, whereas
    StratifiedGroupKFold attempts to create folds which preserve the
    percentage of samples for each class as much as possible given the
    constraint of non-overlapping groups between splits.

    Read more in the :ref:`User Guide <cross_validation>`.

    For visualisation of cross-validation behaviour and
    comparison between common scikit-learn split methods
    refer to :ref:`sphx_glr_auto_examples_model_selection_plot_cv_indices.py`

    Parameters
    ----------
    n_splits : int, default=5
        Number of folds. Must be at least 2.

    shuffle : bool, default=False
        Whether to shuffle each class's samples before splitting into batches.
        Note that the samples within each split will not be shuffled.
        This implementation can only shuffle groups that have approximately the
        same y distribution, no global shuffle will be performed.

    random_state : int or RandomState instance, default=None
        When `shuffle` is True, `random_state` affects the ordering of the
        indices, which controls the randomness of each fold for each class.
        Otherwise, leave `random_state` as `None`.
        Pass an int for reproducible output across multiple function calls.
        See :term:`Glossary <random_state>`.

    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import StratifiedGroupKFold
    >>> X = np.ones((17, 2))
    >>> y = np.array([0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    >>> groups = np.array([1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 5, 5, 6, 6, 7, 8, 8])
    >>> sgkf = StratifiedGroupKFold(n_splits=3)
    >>> sgkf.get_n_splits(X, y)
    3
    >>> print(sgkf)
    StratifiedGroupKFold(n_splits=3, random_state=None, shuffle=False)
    >>> for i, (train_index, test_index) in enumerate(sgkf.split(X, y, groups)):
    ...     print(f"Fold {i}:")
    ...     print(f"  Train: index={train_index}")
    ...     print(f"         group={groups[train_index]}")
    ...     print(f"  Test:  index={test_index}")
    ...     print(f"         group={groups[test_index]}")
    Fold 0:
      Train: index=[ 0  1  2  3  7  8  9 10 11 15 16]
             group=[1 1 2 2 4 5 5 5 5 8 8]
      Test:  index=[ 4  5  6 12 13 14]
             group=[3 3 3 6 6 7]
    Fold 1:
      Train: index=[ 4  5  6  7  8  9 10 11 12 13 14]
             group=[3 3 3 4 5 5 5 5 6 6 7]
      Test:  index=[ 0  1  2  3 15 16]
             group=[1 1 2 2 8 8]
    Fold 2:
      Train: index=[ 0  1  2  3  4  5  6 12 13 14 15 16]
             group=[1 1 2 2 3 3 3 6 6 7 8 8]
      Test:  index=[ 7  8  9 10 11]
             group=[4 5 5 5 5]

    Notes
    -----
    The implementation is designed to:

    * Mimic the behavior of StratifiedKFold as much as possible for trivial
      groups (e.g. when each group contains only one sample).
    * Be invariant to class label: relabelling ``y = ["Happy", "Sad"]`` to
      ``y = [1, 0]`` should not change the indices generated.
    * Stratify based on samples as much as possible while keeping
      non-overlapping groups constraint. That means that in some cases when
      there is a small number of groups containing a large number of samples
      the stratification will not be possible and the behavior will be close
      to GroupKFold.

    See also
    --------
    StratifiedKFold: Takes class information into account to build folds which
        retain class distributions (for binary or multiclass classification
        tasks).

    GroupKFold: K-fold iterator variant with non-overlapping groups.
    """

    def __init__(self, n_splits=5, shuffle=False, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def _iter_test_indices(self, X, y, groups):
        # Implementation is based on this kaggle kernel:
        # https://www.kaggle.com/jakubwasikowski/stratified-group-k-fold-cross-validation
        # and is a subject to Apache 2.0 License. You may obtain a copy of the
        # License at http://www.apache.org/licenses/LICENSE-2.0
        # Changelist:
        # - Refactored function to a class following scikit-learn KFold
        #   interface.
        # - Added heuristic for assigning group to the least populated fold in
        #   cases when all other criteria are equal
        # - Swtch from using python ``Counter`` to ``np.unique`` to get class
        #   distribution
        # - Added scikit-learn checks for input: checking that target is binary
        #   or multiclass, checking passed random state, checking that number
        #   of splits is less than number of members in each class, checking
        #   that least populated class has more members than there are splits.
        rng = check_random_state(self.random_state)
        y = np.asarray(y)
        type_of_target_y = type_of_target(y)
        allowed_target_types = ("binary", "multiclass")
        if type_of_target_y not in allowed_target_types:
            raise ValueError(
                "Supported target types are: {}. Got {!r} instead.".format(
                    allowed_target_types, type_of_target_y
                )
            )

        y = column_or_1d(y)
        _, y_inv, y_cnt = np.unique(y, return_inverse=True, return_counts=True)
        # if np.all(self.n_splits > y_cnt):
        #     raise ValueError(
        #         "n_splits=%d cannot be greater than the"
        #         " number of members in each class." % (self.n_splits)
        #     )
        # n_smallest_class = np.min(y_cnt)
        # if self.n_splits > n_smallest_class:
        #     warnings.warn(
        #         "The least populated class in y has only %d"
        #         " members, which is less than n_splits=%d."
        #         % (n_smallest_class, self.n_splits),
        #         UserWarning,
        #     )
        n_classes = len(y_cnt)

        _, groups_inv, groups_cnt = np.unique(
            groups, return_inverse=True, return_counts=True
        )
        y_counts_per_group = np.zeros((len(groups_cnt), n_classes))
        for class_idx, group_idx in zip(y_inv, groups_inv):
            y_counts_per_group[group_idx, class_idx] += 1

        y_counts_per_fold = np.zeros((self.n_splits, n_classes))
        groups_per_fold = defaultdict(set)

        if self.shuffle:
            rng.shuffle(y_counts_per_group)

        # Stable sort to keep shuffled order for groups with the same
        # class distribution variance
        sorted_groups_idx = np.argsort(
            -np.std(y_counts_per_group, axis=1), kind="mergesort"
        )

        for group_idx in sorted_groups_idx:
            group_y_counts = y_counts_per_group[group_idx]
            best_fold = self._find_best_fold(
                y_counts_per_fold=y_counts_per_fold,
                y_cnt=y_cnt,
                group_y_counts=group_y_counts,
            )
            y_counts_per_fold[best_fold] += group_y_counts
            groups_per_fold[best_fold].add(group_idx)

        for i in range(self.n_splits):
            test_indices = [
                idx
                for idx, group_idx in enumerate(groups_inv)
                if group_idx in groups_per_fold[i]
            ]
            yield test_indices

    def _find_best_fold(self, y_counts_per_fold, y_cnt, group_y_counts):
        best_fold = None
        min_eval = np.inf
        min_samples_in_fold = np.inf
        for i in range(self.n_splits):
            y_counts_per_fold[i] += group_y_counts
            # Summarise the distribution over classes in each proposed fold
            std_per_class = np.std(y_counts_per_fold / y_cnt.reshape(1, -1), axis=0)
            y_counts_per_fold[i] -= group_y_counts
            fold_eval = np.mean(std_per_class)
            samples_in_fold = np.sum(y_counts_per_fold[i])
            is_current_fold_better = (
                fold_eval < min_eval
                or np.isclose(fold_eval, min_eval)
                and samples_in_fold < min_samples_in_fold
            )
            if is_current_fold_better:
                min_eval = fold_eval
                min_samples_in_fold = samples_in_fold
                best_fold = i
        return best_fold

In [10]:
# PreCatatoes
n_splits = 5
n_test_splits = 0
orb = pd.read_csv('/neurospin/dico/data/bv_databases/human/partially_labeled/orbital_patterns/PreCatatoes/OFC_sulcal_type_data_186-subjects_columns-renamed.csv', index_col=0, usecols=['participant_id', 'sex', 'type_OFC_G', 'type_OFC_D '])
orb = orb.dropna()
orb.columns=['Sex', 'Left', 'Right']
orb.index.names = ['Subject']
subset = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/PreCatatoes/crops/2mm/S.Or./mask/Lskeleton_subject.csv')['Subject'].tolist()
orb = orb.loc[subset]

l = orb['Left'].tolist()
for idx, elem in enumerate(l):
    if elem=='I':
        l[idx]=1
    elif elem=='II':
        l[idx]=2
    elif elem=='III':
        l[idx]=3
    elif elem=='IV':
        l[idx]=4
orb['Left'] = l

l = orb['Right'].tolist()
for idx, elem in enumerate(l):
    if elem=='I':
        l[idx]=1
    elif elem=='II':
        l[idx]=2
    elif elem=='III':
        l[idx]=3
    elif elem=='IV':
        l[idx]=4
orb['Right'] = l

In [12]:
orb

,Sex,Left,Right
Subject,,,
101,F,1,2
102,M,2,3
103,M,4,2
104,F,1,3
105,M,2,2
...,...,...,...
95,M,1,3
96,F,3,1
97,M,2,2


In [41]:
# FIP
n_splits=10
n_test_splits = 2
orb = pd.read_excel('/neurospin/dico/data/bv_databases/human/partially_labeled/FIP_patterns/IPS_labels_390.xlsx')
orb = orb.dropna()
orb.columns = ['Subject', 'Sex', 'Left', 'Right']
orb = orb.set_index('Subject')

In [42]:
def print_frequencies(df):
    for right in df['Right'].unique():
        for left in df['Left'].unique():
            for sex in df['Sex'].unique():
                freq = df.query("Right==@right and Left==@left and Sex==@sex")
                print(f"{right}, {left}, {sex}: {len(freq)}")

In [43]:
print_frequencies(orb)

1, 0, M: 44
1, 0, F: 74
1, 1, M: 96
1, 1, F: 56
0, 0, M: 21
0, 0, F: 66
0, 1, M: 12
0, 1, F: 21


In [44]:
def print_results(parent, folds, col, verbose=True):

    # For each conbination of labels, prints the number of rows for each fold
    # having this combination
    total_errors = 0
    n_splits = len(folds)
    if verbose:
        print("query   : #rows      : #rows per fold\n")

    for col0 in parent[col[0]].unique():
        for col1 in parent[col[1]].unique():
            for col2 in parent[col[2]].unique():
                df = parent.query(f"{col[0]}==@col0 and {col[1]}==@col1 and {col[2]}==@col2")
                len_query = len(df)
                if verbose:
                    print(f"{col0}, {col1}, {col2} : total = {len_query} : per fold =", end = ' ')
                for fold in folds:
                    df0 = fold.query(f"{col[0]}==@col0 and {col[1]}==@col1 and {col[2]}==@col2")
                    len_query_fold = len(df0)
                    if abs(len_query_fold-len_query/n_splits) >= 2:
                        total_errors += 1
                    if verbose:
                        print(f"{len_query_fold} -", end= ' ')
                if verbose:
                    print("")

    # Prints the statistics and the number of stratification errors
    expected_total_length = len(parent)
    total_length = 0
    total_mismatches = 0
    print("\nlengths of folds : ", end = ' ')
    for fold in folds:
        len_fold = len(fold)
        print(len_fold, end=' ')
        total_length += len_fold
        if abs(len_fold-expected_total_length/n_splits) >= 2:
            total_mismatches += 1
    print(f"\nExpected total_length = {expected_total_length}")
    print(f"Effective total_length = {total_length}")

    print(f"total number of stratification errors: {total_errors}")
    print(f"total number of mismatched fold sizes : {total_mismatches}")

In [45]:
def iterative_split_through_sorting_shuffle(df, n_splits, stratify_columns, random_state):
    """Custom iterative train test split which
    maintains balanced representation.
    """
    # Dataframe random row shuffle + sorting according to stratify_columns
    sorted = df.sample(frac=1, random_state=random_state).sort_values(stratify_columns)
    # for each fold, we take one row every n_splits rows
    folds = [sorted.iloc[i::n_splits, :] for i in range(n_splits)]
    # Further shuffling
    folds = [fold.sample(frac=1, random_state=random_state) for fold in folds]
    random.Random(random_state).shuffle(folds)
    return folds

In [46]:
folds = iterative_split_through_sorting_shuffle(orb, n_splits, ['Right', 'Left', 'Sex'], 1)

In [47]:
print_results(orb, folds, ['Right', 'Left', 'Sex'])

query   : #rows      : #rows per fold

1, 0, M : total = 44 : per fold = 5 - 4 - 4 - 5 - 5 - 4 - 4 - 5 - 4 - 4 - 
1, 0, F : total = 74 : per fold = 7 - 7 - 7 - 7 - 7 - 8 - 8 - 7 - 8 - 8 - 
1, 1, M : total = 96 : per fold = 10 - 10 - 10 - 10 - 10 - 9 - 9 - 10 - 9 - 9 - 
1, 1, F : total = 56 : per fold = 5 - 6 - 6 - 5 - 5 - 6 - 6 - 5 - 6 - 6 - 
0, 0, M : total = 21 : per fold = 3 - 2 - 2 - 2 - 2 - 2 - 2 - 2 - 2 - 2 - 
0, 0, F : total = 66 : per fold = 6 - 6 - 6 - 6 - 7 - 7 - 7 - 7 - 7 - 7 - 
0, 1, M : total = 12 : per fold = 1 - 2 - 2 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 
0, 1, F : total = 21 : per fold = 2 - 2 - 2 - 3 - 2 - 2 - 2 - 2 - 2 - 2 - 

lengths of folds :  39 39 39 39 39 39 39 39 39 39 
Expected total_length = 390
Effective total_length = 390
total number of stratification errors: 0
total number of mismatched fold sizes : 0


# Save Results

In [48]:
save_path = "/neurospin/dico/data/deep_folding/current/datasets/hcp/FIP"

In [49]:
for i in range(len(folds)):
    folds[i].reset_index()['Subject'].to_csv(
        f"{save_path}/split_{i}.csv",
        header=False,
        index=False)
        #quoting=csv.QUOTE_ALL)

In [50]:
# test csv
l_tot = []
for k in range(len(folds)-n_test_splits):
    df = pd.read_csv(os.path.join(save_path, f'split_{k}.csv'), header=None)
    l = df[0].tolist()
    l_tot += l
df = pd.DataFrame({'Subject': l_tot})
df.to_csv(os.path.join(save_path,'union_splits.csv'), header=None, index=False) #quoting=csv.QUOTE_ALL)

In [51]:
df = orb.reset_index()
df.columns=['Subject', 'Sex', 'Left_FIP', 'Right_FIP']
df.to_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/FIP/FIP_labels.csv', index=False)